In [1]:
from __future__ import division
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%precision 4
plt.style.use('ggplot')

from IPython.core.display import Image
import uuid 

import gc

In [3]:
#Call custom Python module
import parametric as param

In [4]:
def is_Hermitian(m):
    #Set missing to zero prior to checking symmetry
    m[np.isnan(m)] = 0.

    try:
        return np.allclose(np.transpose(m,(1,0)), m)
    except:
        return False

In [1]:
#-------------------------------------------------------------------------------_#
# fastclime_py: Main Function                                                    #
# This Python function has the same functionality as the fastclime function      #
# in the R package, fastclime                                                    # 
# Primary Authors (R, fastclime): Haotian Pang, Han Liu and Robert Vanderbei     #
# Emails: <hpang@princeton.edu>, <hanliu@princeton.edu> and <rvdb@princetonedu>  #
# Adapted for Python by: John Pura                                               #
# Email: <john.pura@duke.edu>                                                    #
#                                                                                #
# Input:
#   x          = 
#   lambda_min = 
#   nlambda    = maximum length of the regularization path to be considered 
#                (default = 50)
#                                                                                #
# Output:                                                                        #
#   cov_input  = scalar                                                          #
#   maxnlambda = scalar representing the length of estimated regularization path # 
#   lambdamtx  = (maxnlambda x d) matrix of column-specific tuning parameters    #
#                for the regularization path                                     #
#                                                                                #
#                                                                                #
#                                                                                #
#                                                                                #
#                                                                                #
#--------------------------------------------------------------------------------#

def fastclime_py(x,lambda_min=0.1,nlambda=50):

    cov_input = 1
    SigmaInput = x.copy()
    
    #Check if matrix is symmetric
    if not is_Hermitian(SigmaInput):
        SigmaInput = np.corrcoef(SigmaInput.T)
        cov_input = 0
    
    #Run parametric simplex linear solver
    Sigmahat, mu, maxnlambda, iicov = param.mainfunc(SigmaInput,lambda_min,nlambda)
      
    #Process output
    maxnlambda+=1
    
    #Reshape the array in Fortran order
    #and then slice the array to extract only the top maxnlambda rows
    lambdamtx = mu.T.reshape(nlambda, -1, order='F')[:maxnlambda,:]
    mu = None
    
    #Take each row of iicov and convert it to a d x d matrix
    d = Sigmahat.shape[1]
    icovlist = np.empty((d, d, maxnlambda)) 
    
    for i in range(maxnlambda):
        icovlist[:,:,i] = iicov[:,i].reshape((d,d)).T
    
    gc.collect()
    
    return x, cov_input, Sigmahat, maxnlambda, lambdamtx, icovlist
    